In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import pandas as pd

# Load the dataset
train_df = pd.read_csv('/Users/anchalbhondekar/Desktop/archive (1)/sign_mnist_train.csv')
test_df = pd.read_csv('/Users/anchalbhondekar/Desktop/archive (1)/sign_mnist_test.csv')

# Split the data into input features (X) and output labels (y)
X_train = train_df.drop('label', axis=1)
y_train = train_df['label']
X_test = test_df.drop('label', axis=1)
y_test = test_df['label']

# Preprocess the data
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# Convert DataFrames to NumPy arrays and reshape
X_train = X_train.values.reshape(-1, 28, 28, 1)
X_test = X_test.values.reshape(-1, 28, 28, 1)

# Define the model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(26, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


858/858 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.3347 - loss: 2.1857 - val_accuracy: 0.7635 - val_loss: 0.6942
Epoch 2/10
858/858 ━━━━━━━━━━━━━━━━━━━━ 590s 688ms/step - accuracy: 0.9212 - loss: 0.2568 - val_accuracy: 0.8654 - val_loss: 0.4760
Epoch 3/10
858/858 ━━━━━━━━━━━━━━━━━━━━ 185s 216ms/step - accuracy: 0.9912 - loss: 0.0404 - val_accuracy: 0.8912 - val_loss: 0.4337
Epoch 4/10
858/858 ━━━━━━━━━━━━━━━━━━━━ 908s 1s/step - accuracy: 0.9945 - loss: 0.0222 - val_accuracy: 0.9018 - val_loss: 0.4437
Epoch 5/10
858/858 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.9979 - loss: 0.0106 - val_accuracy: 0.9106 - val_loss: 0.4273
Epoch 6/10
858/858 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.9985 - loss: 0.0064 - val_accuracy: 0.9013 - val_loss: 0.4118
Epoch 7/10
858/858 ━━━━━━━━━━━━━━━━━━━━ 256s 299ms/step - accuracy: 1.0000 - loss: 0.0013 - val_accuracy: 0.9177 - val_loss: 0.3968
Epoch 8/10
858/858 ━━━━━━━━━━━━━━━━━━━━ 204s 238ms/step - accuracy: 1.0000 - loss: 3.2962e-04 - val_ac

In [ ]:
import numpy as np
import tensorflow as tf
import cv2  # OpenCV for webcam access
from tensorflow.keras.models import Sequential

# Define the model architecture (load it if already trained)
model = Sequential()
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(26, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Load trained weights if available
# model.load_weights('sign_language_model.h5')

# Dictionary to map the predicted class to letters
labels = {i: chr(65 + i) for i in range(26)}

# Function to preprocess frame for model input
def preprocess_frame(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    resized_frame = cv2.resize(gray, (28, 28))  # Resize to 28x28
    input_frame = resized_frame.reshape(1, 28, 28, 1)  # Reshape to fit model input
    input_frame = input_frame.astype('float32') / 255.0  # Normalize pixel values
    return input_frame

# Function to isolate hand region from the background
def hand_segment(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower_skin = np.array([0, 30, 60])
    upper_skin = np.array([20, 150, 255])
    mask = cv2.inRange(hsv, lower_skin, upper_skin)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    hand = cv2.bitwise_and(frame, frame, mask=mask)
    return hand

# Start video capture from the webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()  # Capture frame-by-frame

    if not ret:
        break

    # Isolate hand region
    hand_frame = hand_segment(frame)

    # Preprocess the hand frame
    input_frame = preprocess_frame(hand_frame)

    # Predict the sign using the trained model
    predictions = model.predict(input_frame)
    predicted_class = np.argmax(predictions)
    
    # Display prediction on the frame
    label = labels[predicted_class]
    cv2.putText(frame, f"Prediction: {label}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Show the resulting frame
    cv2.imshow('Sign Language Recognition', frame)

    # Break the loop on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close all windows
cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/st